<h1> Задание:</h1>
Реализовать и обучить Stacked Hourglass Network для поиска ключевых точек лица, а также написать код, который принимает на вход фотографию лица и возвращает выровненное лицо на основе найденных точек

![image.png](https://raw.githubusercontent.com/kjw9899/kjw9899.github.io/master/kjw9899/kjw9899.github.io/assets/images/image-20220331201742790.png)

Конкретные задачи:
* Реализовать Hourglass блок
* Реализовать Stacked Hourglass
* Подготовить датасет, преобразовав точки в Heatmap'ы
* Обучить модель
* Найти или реализовать функцию, которая бы по предсказанным ключевым точкам делала бы выравнивание лица на картинке (face alignment)

<h1> Импорт библиотек</h1>

In [3]:
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import nn

<h1>Hourglass block</h1>

Структура блока:

![image.png](https://i.postimg.cc/fyPxNwn0/images-qsdcfd-post-cac04e49-1042-4492-93fb-bf67f2365fc6-image.png)

Структура выглядит прямо как UNet

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.skip = nn.Identity() if in_channels == out_channels else nn.Conv2d(in_channels, out_channels, 1)

        self.conv1 = nn.Conv2d(in_channels, out_channels // 2, 1)
        self.bn1 = nn.BatchNorm2d(out_channels // 2)
        self.conv2 = nn.Conv2d(out_channels // 2, out_channels // 2, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels // 2)
        self.conv3 = nn.Conv2d(out_channels // 2, out_channels, 1)
        self.bn3 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = self.skip(x)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        return self.relu(x + residual)

In [130]:
class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = ResidualBlock(in_channels, out_channels)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        skip = self.conv(x)
        x = self.pool(skip)
        return x, skip
    

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv = ResidualBlock(in_channels*2, out_channels)
        

    def forward(self, x, skip):
        x = self.upsample(x)
        x = self.conv(torch.cat([x, skip], dim=1))
        return x
    

In [ ]:
class HourglassBlock(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()

        self.down_1 = DownSample(in_channels, features)
        self.down_2 =  DownSample(features, features * 2)
        self.down_3 =  DownSample(features * 2, features * 4)
        self.down_4 =  DownSample(features * 4, features * 8)

        self.bottleneck = ResidualBlock(features, features * 16)        

        self.up_1 = UpSample(features * 16, features * 8)
        self.up_2 = UpSample(features * 8, features * 4)
        self.up_3 = UpSample(features * 4, features * 2)
        self.up_4 = UpSample(features * 2, features)

    def forward(self, x):
        x, skip_1 = self.down_1(x)
        x, skip_2 = self.down_2(x)
        x, skip_3 = self.down_3(x)
        x, skip_4 = self.down_4(x)

        x = self.bottleneck(x)

        x = self.up_1(x, skip_4)
        x = self.up_2(x, skip_3)
        x = self.up_3(x, skip_2)
        x = self.up_4(x, skip_1)

        return x

In [134]:
# чат гпт
import torch.nn as nn

class HourglassBlock(nn.Module):
    def __init__(self, channels, depth):
        super().__init__()
        self.depth = depth

        self.down = DownSample(channels, channels)

        if depth > 1:
            self.inner = HourglassBlock(channels, depth - 1)
        else:
            self.inner = ResidualBlock(channels, channels)

        self.up = UpSample(channels, channels)

    def forward(self, x):
        x_down, skip = self.down(x)
        x_inner = self.inner(x_down)
        x_up = self.up(x_inner, skip)
        return x_up


In [132]:
from torchinfo import summary

model = HourglassBlock()
input_size = (2, 3, 256, 256)  # (batch, channels, height, width)
summary(model, input_size=input_size)

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [DownSample: 1, ResidualBlock: 2, Conv2d: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, MaxPool2d: 2, DownSample: 1, ResidualBlock: 2, Conv2d: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, MaxPool2d: 2, DownSample: 1, ResidualBlock: 2, Conv2d: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, MaxPool2d: 2, DownSample: 1, ResidualBlock: 2, Conv2d: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, Conv2d: 3, BatchNorm2d: 3, ReLU: 3, MaxPool2d: 2, ResidualBlock: 1, Conv2d: 2, Conv2d: 2, BatchNorm2d: 2, ReLU: 2, Conv2d: 2, BatchNorm2d: 2, ReLU: 2, Conv2d: 2, BatchNorm2d: 2, ReLU: 2, Upsample: 2]